In [3]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box

In [4]:
import model
from model_loader import ModelLoader

In [5]:
wd = os.path.dirname(os.path.abspath('__file__'))
weights = os.path.join(wd, 'best.pt')
print(weights)

C:\Users\Willi\Documents\NYU\2020 Spring\Deep Learning\Project\Explore\best.pt


# Test Model Loader

In [6]:
ml = ModelLoader(model_file=weights)

# Sample Some Labeled Data

In [7]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

You will get two different datasets:

 1. an unlabeled dataset for pre-training
 2. a labeled dataset for both training and validation
 
## The dataset is organized into three levels: scene, sample and image

 1. A scene is 25 seconds of a car's journey.
 2. A sample is a snapshot of a scene at a given timeframe. Each scene will be divided into 126 samples, so about 0.2 seconds between consecutive samples.
 3. Each sample contains 6 images captured by camera facing different orientation.
    Each camera will capture 70 degree view. To make it simple, you can safely assume that the angle between the cameras is 60 degrees 

106 scenes in the unlabeled dataset and 28 scenes in the labeled dataset

In [8]:
# You shouldn't change the unlabeled_scene_index
# The first 106 scenes are unlabeled
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)

In [9]:
project_folder = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
provided_folder = os.path.join(project_folder,'Provided_Info')
image_folder = os.path.join(provided_folder,'data')
annotation_csv = os.path.join(image_folder,'annotation.csv')
print(image_folder)

C:\Users\Willi\Documents\NYU\2020 Spring\Deep Learning\Project\Provided_Info\data


In [11]:
transform = torchvision.transforms.ToTensor()

labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )
trainloader = torch.utils.data.DataLoader(labeled_trainset,
                                          batch_size=10,
                                          shuffle=True,
                                          collate_fn=collate_fn)

sample, target, road_image, extra = iter(trainloader).next()
print(torch.stack(sample).shape)

torch.Size([10, 6, 3, 256, 306])


In [22]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()

In [28]:
single = [sample[0][0]]
print(len(single))

1


In [29]:
model.eval()
predictions = model(single)

ImportError: DLL load failed: The specified procedure could not be found.

In [12]:
pred_boxes = ml.get_bounding_boxes(sample)
pred_roads = ml.get_binary_road_map(sample)

ImportError: DLL load failed: The specified procedure could not be found.